<a href="https://colab.research.google.com/github/WangKehanK/CS505/blob/main/Twitter_User_Gender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prediction on age, gender with M3 inference

m3inference

https://github.com/euagendas/m3inference#existing-json-twitter-data

In [1]:
!pip install m3inference

     |████████████████████████████████| 61kB 3.1MB/s 
     |████████████████████████████████| 41.4MB 100kB/s 
  Created wheel for pycld2: filename=pycld2-0.41-cp37-cp37m-linux_x86_64.whl size=9834234 sha256=98cdd789b64a5e1956b22dbd1890d6c9909ace07fe936bd5f5246ad78920b426
  Stored in directory: /root/.cache/pip/wheels/c6/8f/e9/08a1a8932a490175bd140206cd86a3dbcfc70498100de11079
  Created wheel for rauth: filename=rauth-0.7.3-cp37-none-any.whl size=16055 sha256=cdd6560ed378f707d5acb7c51dce4e890bdb40a6a87f5e0b908aa005b7e9a710
  Stored in directory: /root/.cache/pip/wheels/7b/94/5d/81afc278dd5da884a0002563dc4b0fe85f9067a5a40f76f858
Successfully built pycld2 rauth


In [2]:
import numpy as np
import pandas as pd
from m3inference import M3Twitter
import json
import os
import pprint

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df = pd.read_csv("/content/drive/MyDrive/Twitter_user_handles_to_predict.csv", encoding = "utf-8")
df_labeled = pd.read_csv("/content/drive/My Drive/Twitter_users_labeled_with_age_and_gender.csv", encoding = "latin-1")


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25129 entries, 0 to 25128
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   ID        25129 non-null  int64 
 1   Username  25128 non-null  object
dtypes: int64(1), object(1)
memory usage: 392.8+ KB


In [31]:
df_labeled

,Unnamed: 0,screen_name,user_id,lang,name,location,description,protected,followers_count,friends_count,statuses_count,favourites_count,account_created_at,verified,profile_banner_url,profile_background_url,profile_image_url,user.name,num.tweets.used.Lexicon.prediction,Lexicon.age.prediction,Lexicon.gender.prediction..index.,lexicon.gender.prediction,human.labeled.gender,human.labeled.age
0,1,_____zac_____,4.614412e+08,en,zac ¢,"Maryland, USA",_____Û___È_Ü´Ù,False,208,178,27912,4179,1/11/12 20:19,False,https://pbs.twimg.com/profile_banners/46144118...,http://abs.twimg.com/images/themes/theme1/bg.png,http://pbs.twimg.com/profile_images/1226134911...,@_____zac_____,100.0,27.652434,-1.457167,M,NaN,23.0
1,2,___aleia,7.650000e+17,en,_æ___ dad ___æ_,"Ohio, USA",BLACK. LIVES. MATTER.,False,466,388,3313,23454,8/14/16 19:45,False,https://pbs.twimg.com/profile_banners/76491083...,NaN,http://pbs.twimg.com/profile_images/1271280679...,@___aleia,100.0,24.111464,0.985713,F,NaN,19.0
2,3,___Dals,4.361882e+08,it,_ê_ê___ê__ê_ __,NJ,NaN,False,3744,1465,97563,22499,12/13/11 22:07,False,https://pbs.twimg.com/profile_banners/43618822...,http://abs.twimg.com/images/themes/theme14/bg.gif,http://pbs.twimg.com/profile_images/1267852466...,@___Dals,100.0,30.628957,0.734656,F,F,NaN
3,4,___schaeffer___,1.257110e+09,en,Brenden Schaeffer,The Lou,Culver-Stockton College '20 ¢ Ô_Ô_Ô KM 1548...,False,811,660,26670,72205,3/10/13 14:07,False,https://pbs.twimg.com/profile_banners/12571104...,http://abs.twimg.com/images/themes/theme1/bg.png,http://pbs.twimg.com/profile_images/1268044218...,@___schaeffer___,59.0,35.518352,-3.591586,M,NaN,22.0
4,5,__andresiscool,1.030000e+18,en,Andres Navarro,NaN,NaN,False,245,184,8731,18574,8/27/18 5:21,False,https://pbs.twimg.com/profile_banners/10339476...,NaN,http://pbs.twimg.com/profile_images/1272229718...,@__andresiscool,100.0,21.014222,-1.623685,M,M,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3275,3276,ZTheBest33,1.040000e+18,en,Adam. BLM.,northeast elitelism,"AOC/GND/Bern supporter| IFB | very biased | ""W...",False,345,751,17495,33147,9/7/18 11:09,False,https://pbs.twimg.com/profile_banners/10380214...,NaN,http://pbs.twimg.com/profile_images/1232111725...,@ZTheBest33,100.0,37.946711,-2.126250,M,M,NaN
3276,3277,ztran53,3.307187e+09,en,Zachary Tranter,"New York, NY",he/him/his,False,44,148,825,3129,6/3/15 16:58,False,https://pbs.twimg.com/profile_banners/33071871...,http://abs.twimg.com/images/themes/theme1/bg.png,http://pbs.twimg.com/profile_images/6061595328...,@ztran53,100.0,34.509457,-2.469680,M,M,NaN
3277,3278,zupercar1,4.220999e+09,en,Chris Avery _ê___êÈ,"Ann Arbor, MI",Comic Book Artist/ Film Major / I like space.....,False,121,1974,31208,70426,11/18/15 20:57,False,https://pbs.twimg.com/profile_banners/42209994...,http://abs.twimg.com/images/themes/theme1/bg.png,http://pbs.twimg.com/profile_images/1145126143...,@zupercar1,100.0,23.884346,-2.540524,M,NaN,21.0
3278,3279,ZWHITE93,4.193607e+08,en,ZWH!tE,TX_´ÙPA,My life is centered around competition and cur...,False,593,528,69595,20793,11/23/11 8:06,False,https://pbs.twimg.com/profile_banners/41936067...,http://abs.twimg.com/images/themes/theme1/bg.png,http://pbs.twimg.com/profile_images/1229137769...,@ZWHITE93,100.0,32.228124,-0.860006,M,M,NaN


In [32]:
#convert categories to numbers for human.labeled.gender and lexicon.gender.prediction
from sklearn.preprocessing import LabelEncoder
df_labeled = df_labeled[df_labeled['human.labeled.gender'].notna()]
LE = LabelEncoder()
df_labeled['human.labeled.gender'] = LE.fit_transform(df_labeled['human.labeled.gender'])
df_labeled['lexicon.gender.prediction'] = LE.fit_transform(df_labeled['lexicon.gender.prediction'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [33]:
df_labeled

,Unnamed: 0,screen_name,user_id,lang,name,location,description,protected,followers_count,friends_count,statuses_count,favourites_count,account_created_at,verified,profile_banner_url,profile_background_url,profile_image_url,user.name,num.tweets.used.Lexicon.prediction,Lexicon.age.prediction,Lexicon.gender.prediction..index.,lexicon.gender.prediction,human.labeled.gender,human.labeled.age
2,3,___Dals,4.361882e+08,it,_ê_ê___ê__ê_ __,NJ,NaN,False,3744,1465,97563,22499,12/13/11 22:07,False,https://pbs.twimg.com/profile_banners/43618822...,http://abs.twimg.com/images/themes/theme14/bg.gif,http://pbs.twimg.com/profile_images/1267852466...,@___Dals,100.0,30.628957,0.734656,0,0,NaN
4,5,__andresiscool,1.030000e+18,en,Andres Navarro,NaN,NaN,False,245,184,8731,18574,8/27/18 5:21,False,https://pbs.twimg.com/profile_banners/10339476...,NaN,http://pbs.twimg.com/profile_images/1272229718...,@__andresiscool,100.0,21.014222,-1.623685,1,1,NaN
5,6,__blowCeeKisses,9.795642e+07,en,Soleil _´Ù_,"Murda City, MI",Rihanna Stan _Ç___ locÈd queen trusting t...,False,692,754,57586,11811,12/19/09 18:43,False,https://pbs.twimg.com/profile_banners/97956420...,http://abs.twimg.com/images/themes/theme11/bg.gif,http://pbs.twimg.com/profile_images/1274226895...,@__blowCeeKisses,100.0,26.934912,-0.024200,1,0,NaN
6,7,__Bone2,2.276869e+09,en,Bone Beezy_êç,Cleveland,God First! University Of Charleston Alumni HT$...,False,1491,869,16552,24584,1/5/14 0:58,False,https://pbs.twimg.com/profile_banners/22768692...,http://abs.twimg.com/images/themes/theme1/bg.png,http://pbs.twimg.com/profile_images/1267585783...,@__Bone2,100.0,19.886660,-3.282048,1,1,NaN
7,8,__dennisa,3.004387e+09,en,dennise,NaN,NaN,False,143,129,21089,20282,1/29/15 21:47,False,https://pbs.twimg.com/profile_banners/30043869...,http://abs.twimg.com/images/themes/theme1/bg.png,http://pbs.twimg.com/profile_images/1243741019...,@__dennisa,100.0,26.773361,0.531561,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3272,3273,ZoeCalamaco,3.214954e+09,no,Zoe _,San Angelo tx/ aspermont tx,Angelo state,False,475,318,6494,7475,4/28/15 0:24,False,https://pbs.twimg.com/profile_banners/32149543...,http://abs.twimg.com/images/themes/theme1/bg.png,http://pbs.twimg.com/profile_images/1277399838...,@ZoeCalamaco,100.0,21.348766,1.939069,0,0,21.0
3273,3274,zoeynicodemus,1.080000e+18,en,zo _Â,NaN,spread love !!,False,132,139,557,854,1/6/19 20:46,False,https://pbs.twimg.com/profile_banners/10820156...,NaN,http://pbs.twimg.com/profile_images/1277720337...,@zoeynicodemus,100.0,21.543455,0.349508,0,0,NaN
3275,3276,ZTheBest33,1.040000e+18,en,Adam. BLM.,northeast elitelism,"AOC/GND/Bern supporter| IFB | very biased | ""W...",False,345,751,17495,33147,9/7/18 11:09,False,https://pbs.twimg.com/profile_banners/10380214...,NaN,http://pbs.twimg.com/profile_images/1232111725...,@ZTheBest33,100.0,37.946711,-2.126250,1,1,NaN
3276,3277,ztran53,3.307187e+09,en,Zachary Tranter,"New York, NY",he/him/his,False,44,148,825,3129,6/3/15 16:58,False,https://pbs.twimg.com/profile_banners/33071871...,http://abs.twimg.com/images/themes/theme1/bg.png,http://pbs.twimg.com/profile_images/6061595328...,@ztran53,100.0,34.509457,-2.469680,1,1,NaN


In [9]:
df.head()

,ID,Username
0,1,JayHolz410
1,2,kelechief
2,3,VicSpencer
3,4,ItsAlexDodson
4,5,xopinkvodka6


In [11]:
user_list = df_labeled['screen_name'].tolist()
#id_list = df_labeled['user_id'].tolist()
print(user_list)
print(len(user_list))

['___Dals', '__andresiscool', '__blowCeeKisses', '__Bone2', '__dennisa', '__greatperhaps', '__KDA_', '__shawtyx3', '_AJoseph_', '_alfiee11', '_amina_r', '_aniyagraham', '_Baaaates724', '_BattsMan_', '_briannadevaux', '_celiaaaxo', '_CoVeRtToWn_', '_dandysandi', '_DemiNeutron', '_dxVi5', '_elisamtz', '_Eric____', '_espibarrera', '_EWillB', '_Fanaticday_', '_FatThor', '_GabiBozeman', '_Gavin0304', '_gilliansummer_', '_GiuliannaTapia', '_haileyevans', '_Huey93', '_ItsLinaa', '_Jaavvii', '_jameshatfield_', '_jaydezha', '_jonstephen', '_kbatch', '_keagannn', '_Kellyann_C', '_Kingdarrion', '_knweedman', '_lexi_lopezz_', '_liamday', '_lilgremlin', '_madisonage', '_maradelgado', '_megodeck', '_moebettaa', '_mvmacita', '_nikkidaniellee', '_notblakegray', '_olisehh', '_PAULWHISKEY', '_reneejoanne', '_rorr_', '_sacredHeart', '_sebas365', '_selgo_', '_tayloralexis13', '_tayshel_2', '_xogloria', '05Panico', '10oso1996', '1DanLawson', '1grn_eyez', '24_archito', '2no_avail', '2raosanjiv', '307cork', 

In [12]:
username = user_list[0] # try one user first, we can loop over the entire user list to get all the user detail
m3cachedir = "drive/MyDrive/cachedir"

if not os.path.exists(m3cachedir):
  os.mkdir(m3cachedir)
pred = []

In [13]:
# initialization
m3twitter=M3Twitter()

# remember to upload Twitter API keys, format as https://github.com/euagendas/m3inference/blob/master/scripts/auth_example.txt
m3twitter.twitter_init_from_file("/content/drive/MyDrive/auth.txt")

# you can also use m3twitter.infer_id(id = "....")
for i in range(0,len(user_list)):
  print(i)
  user_detail = m3twitter.infer_screen_name(user_list[i],skip_cache=True)
  prob_male = user_detail['output']['gender']['male']
  prob_female = user_detail['output']['gender']['female']
  if prob_male > prob_female:
    pred.append('1')
  else:
    pred.append('0')

04/14/2021 20:26:55 - INFO - m3inference.m3inference -   Version 1.1.4
04/14/2021 20:26:55 - INFO - m3inference.m3inference -   Running on cpu.
04/14/2021 20:26:55 - INFO - m3inference.m3inference -   Will use full M3 model.
04/14/2021 20:26:56 - INFO - m3inference.m3inference -   Dir /root/m3/models/ does not exist. Creating now.
04/14/2021 20:26:56 - INFO - m3inference.m3inference -   Dir /root/m3/models/ created.
04/14/2021 20:26:56 - INFO - m3inference.m3inference -   Model full_model does not exist at /root/m3/models/full_model.mdl. Try to download it now.
04/14/2021 20:26:56 - INFO - m3inference.utils -   /root/m3/models/full_model.mdl not found in cache, downloading from https://nlp.stanford.edu/~zijwang/m3inference/full_model.mdl to /tmp/tmpu59l6d86
141007KB [01:11, 1969.41KB/s]                            
04/14/2021 20:28:08 - INFO - m3inference.utils -   Model full_model was downloaded to a tmp file.
04/14/2021 20:28:08 - INFO - m3inference.utils -   Copying tmp file to /root

0


04/14/2021 20:28:10 - INFO - m3inference.dataset -   1 data entries loaded.
Predicting...: 100%|██████████| 1/1 [00:00<00:00,  1.14it/s]
04/14/2021 20:28:11 - INFO - m3inference.m3twitter -   skip_cache is True. Fetching data from Twitter for __andresiscool.
04/14/2021 20:28:11 - INFO - m3inference.m3twitter -   GET /users/show.json?screen_name=__andresiscool
04/14/2021 20:28:11 - INFO - m3inference.dataset -   1 data entries loaded.
Predicting...:   0%|          | 0/1 [00:00<?, ?it/s]

1


Predicting...: 100%|██████████| 1/1 [00:00<00:00,  1.85it/s]
04/14/2021 20:28:11 - INFO - m3inference.m3twitter -   skip_cache is True. Fetching data from Twitter for __blowceekisses.
04/14/2021 20:28:11 - INFO - m3inference.m3twitter -   GET /users/show.json?screen_name=__blowceekisses
04/14/2021 20:28:11 - INFO - m3inference.dataset -   1 data entries loaded.
Predicting...:   0%|          | 0/1 [00:00<?, ?it/s]

...

04/14/2021 20:58:15 - INFO - m3inference.dataset -   1 data entries loaded.
Predicting...: 100%|██████████| 1/1 [00:00<00:00,  1.62it/s]
04/14/2021 20:58:15 - INFO - m3inference.m3twitter -   skip_cache is True. Fetching data from Twitter for ztran53.
04/14/2021 20:58:15 - INFO - m3inference.m3twitter -   GET /users/show.json?screen_name=ztran53


2142


04/14/2021 20:58:16 - INFO - m3inference.dataset -   1 data entries loaded.
Predicting...: 100%|██████████| 1/1 [00:00<00:00,  1.67it/s]
04/14/2021 20:58:16 - INFO - m3inference.m3twitter -   skip_cache is True. Fetching data from Twitter for zwhite93.
04/14/2021 20:58:16 - INFO - m3inference.m3twitter -   GET /users/show.json?screen_name=zwhite93


2143


04/14/2021 20:58:17 - INFO - m3inference.dataset -   1 data entries loaded.
Predicting...: 100%|██████████| 1/1 [00:00<00:00,  1.67it/s]


In [21]:
count = 0
label = df_labeled['human.labeled.gender'].tolist()
for i in range(len(pred)):
  if int(pred[i]) == label[i]:
    count +=1
print('The accuracy for gender prediction using m3inference is', count/len(pred))

The accuracy for gender prediction using m3inference is 0.8311567164179104


In [16]:
count2 = 0
label1 = df_labeled['lexicon.gender.prediction'].tolist()
for i in range(len(label1)):
  if label1[i] == label[i]:
    count2 +=1
print('The accuracy for lexicon.gender.prediction is', count2/len(label1))

The accuracy for lexicon.gender.prediction is 0.7798507462686567


In [38]:
df_labeled = pd.read_csv("/content/drive/My Drive/Twitter_users_labeled_with_age_and_gender.csv", encoding = "latin-1")
df_labeled = df_labeled[df_labeled['human.labeled.gender'].notna()]
df_labeled["gender"] = "M"
count = 0
for index, row in df_labeled.iterrows():
  if pred[count] == '0':
    df_labeled.at[index, 'gender'] = 'F'
  count+=1
df_labeled

,Unnamed: 0,screen_name,user_id,lang,name,location,description,protected,followers_count,friends_count,statuses_count,favourites_count,account_created_at,verified,profile_banner_url,profile_background_url,profile_image_url,user.name,num.tweets.used.Lexicon.prediction,Lexicon.age.prediction,Lexicon.gender.prediction..index.,lexicon.gender.prediction,human.labeled.gender,human.labeled.age,gender
2,3,___Dals,4.361882e+08,it,_ê_ê___ê__ê_ __,NJ,NaN,False,3744,1465,97563,22499,12/13/11 22:07,False,https://pbs.twimg.com/profile_banners/43618822...,http://abs.twimg.com/images/themes/theme14/bg.gif,http://pbs.twimg.com/profile_images/1267852466...,@___Dals,100.0,30.628957,0.734656,F,F,NaN,F
4,5,__andresiscool,1.030000e+18,en,Andres Navarro,NaN,NaN,False,245,184,8731,18574,8/27/18 5:21,False,https://pbs.twimg.com/profile_banners/10339476...,NaN,http://pbs.twimg.com/profile_images/1272229718...,@__andresiscool,100.0,21.014222,-1.623685,M,M,NaN,M
5,6,__blowCeeKisses,9.795642e+07,en,Soleil _´Ù_,"Murda City, MI",Rihanna Stan _Ç___ locÈd queen trusting t...,False,692,754,57586,11811,12/19/09 18:43,False,https://pbs.twimg.com/profile_banners/97956420...,http://abs.twimg.com/images/themes/theme11/bg.gif,http://pbs.twimg.com/profile_images/1274226895...,@__blowCeeKisses,100.0,26.934912,-0.024200,M,F,NaN,F
6,7,__Bone2,2.276869e+09,en,Bone Beezy_êç,Cleveland,God First! University Of Charleston Alumni HT$...,False,1491,869,16552,24584,1/5/14 0:58,False,https://pbs.twimg.com/profile_banners/22768692...,http://abs.twimg.com/images/themes/theme1/bg.png,http://pbs.twimg.com/profile_images/1267585783...,@__Bone2,100.0,19.886660,-3.282048,M,M,NaN,M
7,8,__dennisa,3.004387e+09,en,dennise,NaN,NaN,False,143,129,21089,20282,1/29/15 21:47,False,https://pbs.twimg.com/profile_banners/30043869...,http://abs.twimg.com/images/themes/theme1/bg.png,http://pbs.twimg.com/profile_images/1243741019...,@__dennisa,100.0,26.773361,0.531561,F,F,NaN,F
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3272,3273,ZoeCalamaco,3.214954e+09,no,Zoe _,San Angelo tx/ aspermont tx,Angelo state,False,475,318,6494,7475,4/28/15 0:24,False,https://pbs.twimg.com/profile_banners/32149543...,http://abs.twimg.com/images/themes/theme1/bg.png,http://pbs.twimg.com/profile_images/1277399838...,@ZoeCalamaco,100.0,21.348766,1.939069,F,F,21.0,F
3273,3274,zoeynicodemus,1.080000e+18,en,zo _Â,NaN,spread love !!,False,132,139,557,854,1/6/19 20:46,False,https://pbs.twimg.com/profile_banners/10820156...,NaN,http://pbs.twimg.com/profile_images/1277720337...,@zoeynicodemus,100.0,21.543455,0.349508,F,F,NaN,F
3275,3276,ZTheBest33,1.040000e+18,en,Adam. BLM.,northeast elitelism,"AOC/GND/Bern supporter| IFB | very biased | ""W...",False,345,751,17495,33147,9/7/18 11:09,False,https://pbs.twimg.com/profile_banners/10380214...,NaN,http://pbs.twimg.com/profile_images/1232111725...,@ZTheBest33,100.0,37.946711,-2.126250,M,M,NaN,M
3276,3277,ztran53,3.307187e+09,en,Zachary Tranter,"New York, NY",he/him/his,False,44,148,825,3129,6/3/15 16:58,False,https://pbs.twimg.com/profile_banners/33071871...,http://abs.twimg.com/images/themes/theme1/bg.png,http://pbs.twimg.com/profile_images/6061595328...,@ztran53,100.0,34.509457,-2.469680,M,M,NaN,M


In [39]:
df_labeled.to_csv('Twitter_users_labeled_prediction_gender.csv', index=True, header=True)

In [71]:
df_labeled2 = pd.read_csv("/content/drive/My Drive/Twitter_users_labeled_with_age_and_gender.csv", encoding = "latin-1")
df_labeled2 = df_labeled.merge(df_labeled2, how='right')
df_labeled2['gender'] = df_labeled2['gender'].fillna('M')
df_labeled2
df_labeled2.to_csv('Twitter_users_labeled_prediction_gender_allUser.csv', index=True, header=True)

In [5]:
user_list2 = df['Username'].tolist()
print(len(user_list2))
print(user_list2[:10])
m3cachedir = "drive/MyDrive/cachedir"

if not os.path.exists(m3cachedir):
  os.mkdir(m3cachedir)
pred2 = []

25129
['JayHolz410', 'kelechief', 'VicSpencer', 'ItsAlexDodson', 'xopinkvodka6', 'larryislegend', 'gucci1017', 'charles270', 'dirtbaglife', 'TaylorJTakeover']


In [33]:
# initialization
m3twitter=M3Twitter()

# remember to upload Twitter API keys, format as https://github.com/euagendas/m3inference/blob/master/scripts/auth_example.txt
m3twitter.twitter_init_from_file("/content/drive/MyDrive/auth.txt")

# you can also use m3twitter.infer_id(id = "....")
for i in range(0,len(user_list2)):
  print(i)
  user_detail = m3twitter.infer_screen_name(user_list2[i],skip_cache=True)
  prob_male = user_detail['output']['gender']['male']
  prob_female = user_detail['output']['gender']['female']
  if prob_male > prob_female:
    pred2.append('1')
  else:
    pred2.append('0')
  if i % 500 == 0:
    print('saved')
    with open("/content/drive/My Drive/pred.txt", "wb") as fp:
      pickle.dump(pred2, fp)


04/15/2021 12:45:57 - INFO - m3inference.m3inference -   Version 1.1.4
04/15/2021 12:45:57 - INFO - m3inference.m3inference -   Running on cpu.
04/15/2021 12:45:57 - INFO - m3inference.m3inference -   Will use full M3 model.
04/15/2021 12:45:57 - INFO - m3inference.m3inference -   Model full_model exists at /root/m3/models/full_model.mdl.
04/15/2021 12:45:57 - INFO - m3inference.utils -   Checking MD5 for model full_model at /root/m3/models/full_model.mdl
04/15/2021 12:45:58 - INFO - m3inference.utils -   MD5s match.
04/15/2021 12:45:58 - INFO - m3inference.m3inference -   Loaded pretrained weight at /root/m3/models/full_model.mdl
04/15/2021 12:45:58 - INFO - m3inference.m3twitter -   skip_cache is True. Fetching data from Twitter for troyfromchicago.
04/15/2021 12:45:58 - INFO - m3inference.m3twitter -   GET /users/show.json?screen_name=troyfromchicago


24917


04/15/2021 12:45:58 - INFO - m3inference.dataset -   1 data entries loaded.


Predicting...:   0%|          | 0/1 [00:00<?, ?it/s]

Predicting...: 100%|██████████| 1/1 [00:00<00:00,  1.44it/s]
04/15/2021 12:45:59 - INFO - m3inference.m3twitter -   skip_cache is True. Fetching data from Twitter for bangbangb0nds.
04/15/2021 12:45:59 - INFO - m3inference.m3twitter -   GET /users/show.json?screen_name=bangbangb0nds
04/15/2021 12:45:59 - WARNING - m3inference.m3twitter -   Could not retreive screen_name
04/15/2021 12:45:59 - WARNING - m3inference.m3twitter -   Could not retreive id_str
04/15/2021 12:45:59 - WARNING - m3inference.m3twitter -   Could not retreive description
04/15/2021 12:45:59 - WARNING - m3inference.m3twitter -   Could not retreive name
04/15/2021 12:45:59 - WARNING - m3inference.m3twitter -   Could not retreive profile_image_url
04/15/2021 12:45:59 - WARNING - m3inference.m3twitter -   Unable to extract image from Twitter. Using default image.
04/15/2021 12:45:59 - INFO -

24918




Predicting...: 100%|██████████| 1/1 [00:00<00:00,  1.48it/s]
04/15/2021 12:46:00 - INFO - m3inference.m3twitter -   skip_cache is True. Fetching data from Twitter for 6creamo.
04/15/2021 12:46:00 - INFO - m3inference.m3twitter -   GET /users/show.json?screen_name=6creamo
04/15/2021 12:46:00 - WARNING - m3inference.m3twitter -   Could not retreive screen_name
04/15/2021 12:46:00 - WARNING - m3inference.m3twitter -   Could not retreive id_str
04/15/2021 12:46:00 - WARNING - m3inference.m3twitter -   Could not retreive description
04/15/2021 12:46:00 - WARNING - m3inference.m3twitter -   Could not retreive name
04/15/2021 12:46:00 - WARNING - m3inference.m3twitter -   Could not retreive profile_image_url
04/15/2021 12:46:00 - WARNING - m3inference.m3twitter -   Unable to extract image from Twitter. Using default image.
04/15/2021 12:46:00 - INFO - m3inference.dataset -   1 data entries loaded.


Predicting...:   0%|          | 0/1 [00:00<?, ?it/s]

...

04/15/2021 12:48:45 - INFO - m3inference.dataset -   1 data entries loaded.


Predicting...:   0%|          | 0/1 [00:00<?, ?it/s]

Predicting...: 100%|██████████| 1/1 [00:00<00:00,  1.67it/s]
04/15/2021 12:48:46 - INFO - m3inference.m3twitter -   skip_cache is True. Fetching data from Twitter for oghenenerojr.
04/15/2021 12:48:46 - INFO - m3inference.m3twitter -   GET /users/show.json?screen_name=oghenenerojr


25127


04/15/2021 12:48:46 - INFO - m3inference.dataset -   1 data entries loaded.


Predicting...:   0%|          | 0/1 [00:00<?, ?it/s]

Predicting...: 100%|██████████| 1/1 [00:00<00:00,  1.53it/s]
04/15/2021 12:48:47 - INFO - m3inference.m3twitter -   skip_cache is True. Fetching data from Twitter for earlxsweat.
04/15/2021 12:48:47 - INFO - m3inference.m3twitter -   GET /users/show.json?screen_name=earlxsweat
04/15/2021 12:48:47 - INFO - m3inference.dataset -   1 data entries loaded.


Predicting...:   0%|          | 0/1 [00:00<?, ?it/s]

25128




Predicting...: 100%|██████████| 1/1 [00:00<00:00,  1.64it/s]


In [34]:
print(len(pred2))

25129


In [35]:
import pickle
with open("/content/drive/My Drive/pred.txt", "wb") as fp:
  pickle.dump(pred2, fp)


In [43]:
import pickle
with open("/content/drive/My Drive/pred.txt", "rb") as fp:   # Unpickling
  ls = pickle.load(fp)
print(len(ls))
pred2[-10:]

25129


['1', '1', '1', '1', '0', '1', '1', '1', '1', '1']

In [40]:
df["gender"] = "M"

for index in range(len(df)):
  if pred2[index] == '0':
    df.at[index, 'gender'] = 'F'

df.head()
df.to_csv('Twitter_user_handles_to_predict_gender.csv', index=True, header=True)

In [41]:
df

,ID,Username,gender
0,1,JayHolz410,M
1,2,kelechief,M
2,3,VicSpencer,M
3,4,ItsAlexDodson,F
4,5,xopinkvodka6,M
...,...,...,...
25124,25125,13luvr,M
25125,25126,Racetac,M
25126,25127,MrNoFilter_100,M
25127,25128,OgheneNerojr,M
